# Configuration and Utilities


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from matplotlib.dates import date2num
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras

import re
import math

In [ ]:
# Date parsing
def dateparse (time_in_secs):    
    return datetime.fromtimestamp(float(time_in_secs))

In [ ]:
def highlights_attacks (df, attacks, ax):
  for index, row in attacks.iterrows():
    start = datetime.fromtimestamp(row['start'])
    end = datetime.fromtimestamp(row['end'])

    period = df[(df.index > start) & (df.index <= end)].index

    ax.axvspan(start, end, facecolor='pink', edgecolor='none', alpha=.4)

In [ ]:
def clean_header (header_name, label):
    x = re.match(r"\(\'value\',\s\'(\w+)\-\S*\-\S*\'\)", header_name)
    if x == None:
        return header_name
    return x.group(1) + '-' + label


In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs).astype('float32'), np.array(ys).astype('float32')

In [ ]:
def return_rmse(test, predicted, model):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error for {} model is {}.".format(model, rmse))
    return rmse

In [ ]:
attacks = pd.DataFrame(
    {
        "name": ['First Attack'],
        "start" : [1619499815],
        "end" : [1619499840]
    }
)

# Fetching datasets

## Container Traffic Dataset



### Container Traffic read and clean

In [ ]:
container_traffic = pd.read_csv('/kaggle/input/chaosml/pod_traffic.csv', parse_dates=['timestamp'], date_parser=dateparse)
container_traffic.head()

In [ ]:
container_traffic_pivoted = pd.pivot_table(data=container_traffic,index=['timestamp'], columns=['pod'])
container_traffic_pivoted.head()

In [ ]:
container_traffic_flattened = pd.DataFrame(container_traffic_pivoted.to_records())
container_traffic_flattened.columns = [clean_header(hdr,'threads') for hdr in container_traffic_flattened.columns]
container_traffic_flattened.head()

In [ ]:
# may move after merge
container_traffic_flattened['attack'] = False

for index, row in attacks.iterrows():
  start = datetime.fromtimestamp(row['start'])
  end = datetime.fromtimestamp(row['end'])
  container_traffic_flattened.loc[(container_traffic_flattened['timestamp'] > start) & (container_traffic_flattened['timestamp'] <= end), 'attack'] = True

container_traffic_flattened.head()

### Container Traffic Plot

In [ ]:
indexed_container_traffic_flattened = container_traffic_flattened
indexed_container_traffic_flattened.set_index('timestamp', inplace=True)

In [ ]:
ax = indexed_container_traffic_flattened.plot()

highlights_attacks (indexed_container_traffic_flattened, attacks, ax)


## Container Satauration CPU Dataset

### Container Saturation read and clean

In [ ]:
container_saturation_cpu = pd.read_csv('/kaggle/input/chaosml/pod_saturation_cpu.csv', parse_dates=['timestamp'], date_parser=dateparse)
container_saturation_cpu.head()

In [ ]:
container_saturation_cpu_pivoted = pd.pivot_table(data=container_saturation_cpu,index=['timestamp'], columns=['pod'])
container_traffic_pivoted.head()

In [ ]:
container_saturation_cpu_flattened = pd.DataFrame(container_saturation_cpu_pivoted.to_records())
container_saturation_cpu_flattened.columns = [clean_header(hdr,'cpu') for hdr in container_saturation_cpu_flattened.columns]
container_saturation_cpu_flattened.head()

### Container Saturation CPU Plot

In [ ]:
indexed_container_saturation_cpu_flattened = container_saturation_cpu_flattened
indexed_container_saturation_cpu_flattened.set_index('timestamp', inplace=True)

In [ ]:
ax = indexed_container_saturation_cpu_flattened.plot()

highlights_attacks (indexed_container_saturation_cpu_flattened, attacks, ax)

## Container Saturation Memory dataset

In [ ]:
container_saturation_mem = pd.read_csv('/kaggle/input/chaosml/pod_saturation_memory.csv', parse_dates=['timestamp'], date_parser=dateparse)
container_saturation_mem.head()

In [ ]:
container_saturation_mem_pivoted = pd.pivot_table(data=container_saturation_mem,index=['timestamp'], columns=['pod'])
container_saturation_mem_pivoted.head()

In [ ]:
container_saturation_mem_flattened = pd.DataFrame(container_saturation_mem_pivoted.to_records())
container_saturation_mem_flattened.columns = [clean_header(hdr,'memory') for hdr in container_saturation_mem_flattened.columns]
container_saturation_mem_flattened.head()

### Container Saturation Memory plot

In [ ]:
indexed_container_saturation_mem_flattened = container_saturation_mem_flattened
indexed_container_saturation_mem_flattened.set_index('timestamp', inplace=True)

In [ ]:
ax = indexed_container_saturation_mem_flattened.plot()

highlights_attacks (indexed_container_saturation_mem_flattened, attacks, ax)


# Merge Datasets

Documentation about `merge_asof` function may be found [here](https://towardsdatascience.com/how-to-merge-not-matching-time-series-with-pandas-7993fcbce063)

In [ ]:
merged_traffic_and_cpu = pd.merge_asof (container_traffic_flattened, container_saturation_cpu_flattened, on='timestamp', tolerance=pd.Timedelta('1s'))
merged_traffic_and_cpu.head()

In [ ]:
merged_traffuc_cpu_and_mem = pd.merge_asof (merged_traffic_and_cpu, container_saturation_mem_flattened, on='timestamp', tolerance=pd.Timedelta('1s'))
merged_traffuc_cpu_and_mem.head()

In [ ]:
merged_traffuc_cpu_and_mem.describe().transpose()

## Replace Missing Values

In [ ]:
# replace with last value

colums_to_replace_with_last_value = ['cartservice-cpu', 'currencyservice-cpu', 'checkoutservice-cpu', 'adservice-cpu', 'cartservice-cpu', 'checkoutservice-cpu',
                                     'currencyservice-cpu', 'emailservice-cpu', 'frontend-cpu', 'loadgenerator-cpu', 'paymentservice-cpu', 'productcatalogservice-cpu',
                                     'recommendationservice-cpu', 'redis-cpu', 'shippingservice-cpu', 'adservice-memory','cartservice-memory', 'checkoutservice-memory',
                                     'currencyservice-memory', 'emailservice-memory', 'frontend-memory', 'loadgenerator-memory', 'paymentservice-memory',
                                     'productcatalogservice-memory', 'recommendationservice-memory', 'redis-memory', 'shippingservice-memory']

for column in colums_to_replace_with_last_value:
    merged_traffuc_cpu_and_mem[column].replace(np.nan, merged_traffuc_cpu_and_mem[column].mean(), inplace=True)

    
# replace with max value

columns_to_replace_with_max_val = []

for column in columns_to_replace_with_max_val:
    merged_traffuc_cpu_and_mem[column].replace(np.nan, merged_traffuc_cpu_and_mem[column].max(), inplace=True)

In [ ]:
merged_traffuc_cpu_and_mem.isnull().sum(axis = 0)

### Merge Plot

In [ ]:
indexed_merge = merged_traffuc_cpu_and_mem
indexed_merge.set_index('timestamp', inplace=True)

In [ ]:
indexed_merge.plot(subplots=True, y=['frontend-threads', 'frontend-cpu', 'frontend-memory'])

# Train a model

In [ ]:
f_predicted = 'frontend-cpu' #'frontend-memory'

In [ ]:
epochs_number = 30

In [ ]:
learning_rate = 0.01

regularizers = [
    keras.regularizers.L1L2(l1=0.1, l2=0.1),
    keras.regularizers.L1L2(l1=0.01, l2=0.01),
    keras.regularizers.L1L2(l1=0.5, l2=0.5),
    keras.regularizers.L1L2(l1=0.05, l2=0.05)
]

## Pre processing

We’ll use the last 20% of the data for testing, the other 80% will be splitted in 60% for trainning and 20% for validation

In [ ]:
train_and_validation_percentaje = 0.9
validation_percentaje = 0.889

train_and_validation_size = int(len(indexed_merge) * train_and_validation_percentaje)
test_size = len(indexed_merge) - train_and_validation_size

train_and_validation, test = indexed_merge.iloc[0:train_and_validation_size], indexed_merge.iloc[train_and_validation_size:len(indexed_merge)]

print(len(train_and_validation), len(test))

train_size = int(len(train_and_validation)* validation_percentaje)
validation_size = len(train_and_validation) - train_size

train, validation = train_and_validation.iloc[0:train_size], train_and_validation.iloc[train_size:len(train_and_validation)]

print(len(train), len(validation), len(test))

We’ll scale some of the features we’re using for our modeling:

In [ ]:
f_columns = indexed_merge.columns.to_list()
f_columns.remove('attack')
f_columns.remove(f_predicted)

f_transformer = RobustScaler()
v_transformer = RobustScaler()
frontend_memory_transformer = RobustScaler()

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
v_transformer = v_transformer.fit(validation[f_columns].to_numpy())
frontend_memory_transformer = frontend_memory_transformer.fit(train[[f_predicted]])

train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
train[f_predicted] = frontend_memory_transformer.transform(train[[f_predicted]])

validation.loc[:, f_columns] = v_transformer.transform(validation[f_columns].to_numpy())
validation[f_predicted] = frontend_memory_transformer.transform(validation[[f_predicted]])

test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())
test[f_predicted] = frontend_memory_transformer.transform(test[[f_predicted]])

print(len(train), len(validation), len(test))
print(train.shape, validation.shape, test.shape)

In [ ]:
time_steps = 10

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train[f_predicted], time_steps)
X_validation, y_validation = create_dataset(validation, validation[f_predicted], time_steps)
X_test, y_test = create_dataset(test, test[f_predicted], time_steps)

print(X_train.shape, y_train.shape)
print(X_validation.shape, y_validation.shape)
print(X_test.shape, y_test.shape)

## Prediction with LSTM

In [ ]:
lstm_models = []
for reg in regularizers:
    model = keras.Sequential()
    model.add(
      keras.layers.LSTM(
        units=128,
        input_shape=(X_train.shape[1], X_train.shape[2]),
        kernel_regularizer=reg
      )
    )
    model.add(keras.layers.Dropout(rate=0.2))

    model.add(keras.layers.Dense(units=1))
    opti = keras.optimizers.RMSprop(learning_rate=learning_rate)
    model.compile(optimizer=opti,loss='mean_squared_error')
    
    lstm_models.append(model)

In [ ]:
for model in lstm_models:
    history_lstm = model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=32,
        shuffle=False,
        # We pass some validation for
        # monitoring validation loss and metrics
        # at the end of each epoch
        validation_data=(X_validation, y_validation),
    )

In [ ]:
lstm_models[0].summary()

In [ ]:
y_pred_lstm = []
y_pred_lstm_val = []

for model in lstm_models:
    y_pred_lstm.append(model.predict(X_test))
    y_pred_lstm_val.append(model.predict(X_validation))

In [ ]:
y_pred_inv_lstm = []
y_pred_inv_lstm_val = []

y_train_inv = frontend_memory_transformer.inverse_transform(y_train.reshape(1, -1))
y_validation_inv = frontend_memory_transformer.inverse_transform(y_validation.reshape(1, -1))
y_test_inv = frontend_memory_transformer.inverse_transform(y_test.reshape(1, -1))

for i, _ in enumerate(lstm_models):
    y_pred_inv_lstm.append(frontend_memory_transformer.inverse_transform(y_pred_lstm[i]))
    y_pred_inv_lstm_val.append(frontend_memory_transformer.inverse_transform(y_pred_lstm_val[i]))

In [ ]:
fig, axs = plt.subplots(2, 2)
fig.suptitle('LSTM')

regu_titles = [
    '(L1=0.1, L2=0.1)', '(L1=0.01, L2=0.01)',
    '(L1=0.5, L2=0.5)', '(L1=0.05, L2=0.05)'
]

for i, model in enumerate(lstm_models):
    x_view = i % 2
    y_view = 0 if i < 2 else 1
    axs[x_view, y_view].plot(np.arange(0, len(y_train)), y_train_inv.flatten(), 'g', label="history")
    axs[x_view, y_view].plot(np.arange(len(y_train), len(y_train) + len(y_validation)), y_validation_inv.flatten(), 'y', label="validation")
    axs[x_view, y_view].plot(np.arange(len(y_train) + len(y_validation), len(y_train) + len(y_validation) + len(y_test)), y_test_inv.flatten(), label="true")
    axs[x_view, y_view].plot(np.arange(len(y_train) + len(y_validation), len(y_train) + len(y_validation) + len(y_test)), y_pred_inv_lstm[i].flatten(), 'r', label="prediction")
    axs[x_view, y_view].set_title(regu_titles[i])

for ax in axs.flat:
    ax.set(xlabel='Time Step', ylabel='Frontend CPU')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

In [ ]:
rmse_lstm_val = []

for i, _ in enumerate(lstm_models):
    rmse_lstm_val.append(return_rmse(y_validation,y_pred_inv_lstm_val[i],'LSTM'))

In [ ]:
rmse_lstm = []

for i, _ in enumerate(lstm_models):
    rmse_lstm.append(return_rmse(y_test,y_pred_inv_lstm[i],'LSTM'))

## Prediction with Bidirectional LSTM with a Dropout layer

In [ ]:
blstm_models = []

for reg in regularizers:
    model = keras.Sequential()

    model.add(
      keras.layers.Bidirectional(
        keras.layers.LSTM(
          units=50,
          input_shape=(X_train.shape[1], X_train.shape[2]),
          kernel_regularizer=reg
        )
      )
    )
    model.add(keras.layers.Dropout(rate=0.2))

    model.add(keras.layers.Dense(units=1))

    opti = keras.optimizers.RMSprop(learning_rate=learning_rate)
    model.compile(optimizer=opti,loss='mean_squared_error')
    
    blstm_models.append(model)

In [ ]:
for model in blstm_models:
    history_blstm = model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=32,
        shuffle=False,
        # We pass some validation for
        # monitoring validation loss and metrics
        # at the end of each epoch
        validation_data=(X_validation, y_validation),
    )

In [ ]:
blstm_models[0].summary()

In [ ]:
y_pred_blstm = []
y_pred_blstm_val = []

for model in blstm_models:
    y_pred_blstm.append(model.predict(X_test))
    y_pred_blstm_val.append(model.predict(X_validation))

Reverse scaling

In [ ]:
y_pred_inv_blstm = []
y_pred_inv_blstm_val = []

y_train_inv = frontend_memory_transformer.inverse_transform(y_train.reshape(1, -1))
y_validation_inv = frontend_memory_transformer.inverse_transform(y_validation.reshape(1, -1))
y_test_inv = frontend_memory_transformer.inverse_transform(y_test.reshape(1, -1))

for i, _ in enumerate(blstm_models):
    y_pred_inv_blstm.append(frontend_memory_transformer.inverse_transform(y_pred_blstm[i]))
    y_pred_inv_blstm_val.append(frontend_memory_transformer.inverse_transform(y_pred_blstm_val[i]))

In [ ]:
fig, axs = plt.subplots(2, 2)
fig.suptitle('Bidirectional LSTM')

regu_titles = [
    '(L1=0.1, L2=0.1)', '(L1=0.01, L2=0.01)',
    '(L1=0.5, L2=0.5)', '(L1=0.05, L2=0.05)'
]

for i, model in enumerate(blstm_models):
    x_view = i % 2
    y_view = 0 if i < 2 else 1
    axs[x_view, y_view].plot(np.arange(0, len(y_train)), y_train_inv.flatten(), 'g', label="history")
    axs[x_view, y_view].plot(np.arange(len(y_train), len(y_train) + len(y_validation)), y_validation_inv.flatten(), 'y', label="validation")
    axs[x_view, y_view].plot(np.arange(len(y_train) + len(y_validation), len(y_train) + len(y_validation) + len(y_test)), y_test_inv.flatten(), label="true")
    axs[x_view, y_view].plot(np.arange(len(y_train) + len(y_validation), len(y_train) + len(y_validation) + len(y_test)), y_pred_inv_blstm[i].flatten(), 'r', label="prediction")
    axs[x_view, y_view].set_title(regu_titles[i])

for ax in axs.flat:
    ax.set(xlabel='Time Step', ylabel='Frontend CPU')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

In [ ]:
rmse_blstm_val = []

for i, _ in enumerate(blstm_models):
    rmse_blstm_val.append(return_rmse(y_validation, y_pred_inv_blstm_val[i],'Bidirectional LSTM'))

In [ ]:
rmse_blstm = []

for i, _ in enumerate(blstm_models):
    rmse_blstm.append(return_rmse(y_test, y_pred_inv_blstm[i],'Bidirectional LSTM'))

## Prediction with GRU

In [ ]:
gru_models = []

for reg in regularizers:
    model = keras.Sequential()

    model.add(
      keras.layers.GRU(
        units=50,
        input_shape=(X_train.shape[1], X_train.shape[2]),
        activation='tanh',
        kernel_regularizer=reg
      )
    )
    model.add(keras.layers.Dropout(rate=0.2))

    model.add(keras.layers.Dense(units=1))

    model.compile(optimizer=keras.optimizers.SGD(lr=learning_rate, decay=1e-7, momentum=0.9, nesterov=False), loss='mean_squared_error')
    
    gru_models.append(model)

In [ ]:
for model in gru_models:
    history_gru = model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=150,
        shuffle=False,
        # We pass some validation for
        # monitoring validation loss and metrics
        # at the end of each epoch
        validation_data=(X_validation, y_validation),
    )

In [ ]:
gru_models[0].summary()

In [ ]:
y_pred_gru = []
y_pred_gru_val = []

for model in gru_models:
    y_pred_gru.append(model.predict(X_test))
    y_pred_gru_val.append(model.predict(X_validation))

In [ ]:
y_pred_inv_gru = []
y_pred_inv_gru_val = []

y_train_inv = frontend_memory_transformer.inverse_transform(y_train.reshape(1, -1))
y_validation_inv = frontend_memory_transformer.inverse_transform(y_validation.reshape(1, -1))
y_test_inv = frontend_memory_transformer.inverse_transform(y_test.reshape(1, -1))

for i, _ in enumerate(blstm_models):
    y_pred_inv_gru.append(frontend_memory_transformer.inverse_transform(y_pred_gru[i]))
    y_pred_inv_gru_val.append(frontend_memory_transformer.inverse_transform(y_pred_gru_val[i]))

In [ ]:
fig, axs = plt.subplots(2, 2)
fig.suptitle('GRU')

regu_titles = [
    '(L1=0.1, L2=0.1)', '(L1=0.01, L2=0.01)',
    '(L1=0.5, L2=0.5)', '(L1=0.05, L2=0.05)'
]

for i, model in enumerate(blstm_models):
    x_view = i % 2
    y_view = 0 if i < 2 else 1
    axs[x_view, y_view].plot(np.arange(0, len(y_train)), y_train_inv.flatten(), 'g', label="history")
    axs[x_view, y_view].plot(np.arange(len(y_train), len(y_train) + len(y_validation)), y_validation_inv.flatten(), 'y', label="validation")
    axs[x_view, y_view].plot(np.arange(len(y_train) + len(y_validation), len(y_train) + len(y_validation) + len(y_test)), y_test_inv.flatten(), label="true")
    axs[x_view, y_view].plot(np.arange(len(y_train) + len(y_validation), len(y_train) + len(y_validation) + len(y_test)), y_pred_inv_gru[i].flatten(), 'r', label="prediction")
    axs[x_view, y_view].set_title(regu_titles[i])

for ax in axs.flat:
    ax.set(xlabel='Time Step', ylabel='Frontend CPU')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

In [ ]:
rmse_gru_val = []

for i, _ in enumerate(gru_models):
    rmse_gru_val.append(return_rmse(y_validation, y_pred_inv_gru_val[i], 'GRU'))

In [ ]:
rmse_gru = []

for i, _ in enumerate(gru_models):
    rmse_gru.append(return_rmse(y_test, y_pred_inv_gru[i], 'GRU'))

## Compare Performance

In [ ]:
regu_index = [
    '(L1=0.1, L2=0.1)', '(L1=0.01, L2=0.01)', '(L1=0.5, L2=0.5)', '(L1=0.05, L2=0.05)'
]

rmse_df = pd.DataFrame({'LSTM': rmse_lstm_val,
                        'BLSTM': rmse_blstm_val,
                        'GRU': rmse_gru_val},
                       index=regu_index)

ax = rmse_df.plot.bar(rot=0, figsize=(15,6))

ax.set_title('Model Comparision with Learning rate of 0.001')
ax.legend(bbox_to_anchor=(1.0, 1.0))

for p in ax.patches:
    b = p.get_bbox()
    val = "{:.2f}".format(b.y1 + b.y0)        
    ax.annotate(val, ((b.x0 + b.x1)/2 - 0.07, b.y1 + 0.005))